# Pelita Notebook
Defining a player and running a Pelita game from inside IPython

First: some basic imports

In [1]:
import pelita
from players import FoodEatingPlayer
from pelita.game_master import GameMaster
from pelita.player import AbstractPlayer, SimpleTeam

The player which we define is the `SmartRandomPlayer`, which is also shown on the homepage. Nothing too fancy here…

In [2]:
from pelita import datamodel
class SmartRandomPlayer(AbstractPlayer):
    def get_move(self):
        dangerous_enemy_pos = [bot.current_pos
            for bot in self.enemy_bots if bot.is_destroyer]
        killable_enemy_pos = [bot.current_pos
            for bot in self.enemy_bots if bot.is_harvester]

        smart_moves = []
        for move, new_pos in list(self.legal_moves.items()):
            if (move == datamodel.stop or
                new_pos in dangerous_enemy_pos):
                continue # bad idea
            elif (new_pos in killable_enemy_pos or
                  new_pos in self.enemy_food):
                return move # get it!
            else:
                smart_moves.append(move)

        if smart_moves:
            return self.rnd.choice(smart_moves)
        else:
            # we ran out of smart moves
            return datamodel.stop

We define our

In [3]:
team = SimpleTeam("My winning team", SmartRandomPlayer(), SmartRandomPlayer())

… and the enemy’s team.

In [4]:
enemy = SimpleTeam("Enemy team", FoodEatingPlayer(), FoodEatingPlayer())

We set-up a maze

In [5]:
layout = """
##################################
#...   #      .#     #  #       3#
# ## #   # ###    #  #     #####1#
#.   # #    # .   # ##           #
#.#    #  .    #    .  # #########
# ## # ## ####    # ##.   . .   .#
#.. .  .  #. . #. #  # ## #####  #
# ## #### #.## #     #  .  . . ..#
#..  ..   # #  #  #    ##### #####
##### #####    #  #  # #   ..  ..#
#.. . .  .  #     # ##.# #### ## #
#  ##### ## #  # .# . .#  .  . ..#
#.   . .   .## #    #### ## # ## #
######### #  .    #    .  #    #.#
#           ## #   . #    # #   .#
#0#####     #  #    ### #   # ## #
#2       #  #     #.      #   ...#
##################################
"""

Now it’s time for some interaction with IPython. Basically, we hijack IPython’s tornado application and use this for our own websockets and small file server.

In [6]:
import pelita.ipython
from IPython.display import IFrame

Register our application:

In [7]:
application, sockets = pelita.ipython.init_app()

Listening on 0.0.0.0, port 52588


This ipc socket is a subscriber sink for a Pelita zmq publisher socket:

In [8]:
application.path

'ipc:///tmp/pelita.4360207608'

We create a publisher which will act as a viewer for the Pelita process and send all data. (Every application should have only one publisher, otherwise it might not work.)

In [9]:
from pelita.simplesetup import SimplePublisher

In [10]:
publisher = SimplePublisher(application.path)

Now we’ll get the HTML output prepared in an iframe. This will accept data over a websocket and start displaying our little game once the data arrives.

In [11]:
url = "http://%s:%i/static/index.html" % sockets[0].getsockname()
print("This iframe is also available as: {url}".format(url=url))
IFrame(url, width=600, height=300)

This iframe is also available as: http://0.0.0.0:52588/static/index.html


Finally, we set up a game with the given layout and teams. Remember to always connect to the publisher before starting the game. Otherwise, there’ll be no output.

In [12]:
gm = GameMaster(layout, [team, enemy], 4, 100)
gm.register_viewer(publisher)
gm.play()

WebSocket opened


The most likely outcome is, that we lose badly against the FoodEatingPlayers.